# Object detection

Object detection + segmentation using ultralytics [YOLO](https://github.com/ultralytics/ultralytics)

In [1]:
import os
import pickle

from tqdm import tqdm
from ultralytics import YOLO

## Performing inference and storing predictions

Predicting using weights of YOLO 11x for segmentation trained on [COCO's](https://cocodataset.org/#home) dataset.

In [6]:
type = 'nat'

model = YOLO('models/segmentation/yolo11x-seg.pt')
model.to('cuda')



results_nat = {}

image_files = [os.path.join(f'images/images_{type}', f) for f in os.listdir(f'images/images_{type}') if os.path.isfile(os.path.join(f'images/images_{type}', f))]
for image_file in tqdm(image_files, total=len(image_files)):
    results[image_file[image_file.rfind('\\')+1:]] = model.predict(image_file, verbose=False)[0]

with open('predictions/saved_predictions.pkl', 'wb') as f:
    pickle.dump(results, f)


100%|██████████| 200/200 [00:32<00:00,  6.23it/s]


## Visualizing Results

In [2]:
with open('predictions/saved_predictions.pkl', 'rb') as f:
    results = pickle.load(f)

results[list(results.keys())[20]].show()

## Saving Masks

In [15]:
results[list(results.keys())[0]].masks[0]

ultralytics.engine.results.Masks object with attributes:

data: tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], device='cuda:0')
orig_shape: (341, 512)
shape: torch.Size([1, 448, 640])
xy: [array([[      161.6,       261.7],
       [      161.6,       273.7],
       [        164,       273.7],
       [      164.8,       274.5],
       [      164.8,       275.3],
       [      165.6,       276.1],
       [      165.6,       278.5],
       [      166.4,       279.3],
       [      166.4,       280.9],
       [      167.2,       281.7],
       [      167.2,       284.1],
       [        168,       284.9],
       [        168,       286.5],
       [      168.8,       287.3],
       [      168.8,       291.3],
       [      169.6,       292.1],
       [      169.6,       296.9],
 

In [10]:
for key in results.keys():
    
    if results[key].masks is None:
        continue                    # MAYBE ADD SOMETHING TO INDICATE THERE'S NO MASK
    else:
        for i in range(len(results[key].masks)):
            print(i)

0
1
2
3
4
5
6
7
8
9
10
0
1
0
1
2
3
4
5
0
1
0
1
0
1
0
0
1
2
3
4
5
0
0
0
1
0
1
0
1
2
0
0
1
2
3
4
5
0
1
0
1
2
3
4
5
0
1
2
0
1
0
1
2
3
4
5
6
0
1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
0
1
2
3
4
5
6
7
8
9
0
0
1
2
3
4
0
1
0
0
0
1
2
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
0
1
2
3
4
5
6
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
8
0
1
2
3
0
0
1
2
3
0
1
2
0
1
2
3
4
5
6
7
8
9
10
11
12
0
0
1
0
1
2
3
4
5
6
7
8
9
10
0
1
0
1
0
1
0
1
2
3
0
1
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
8
9
0
1
0
1
2
3
0
1
2
0
0
1
2
0
1
2
3
4
5
6
7
8
9
0
1
2
0
0
0
1
0
1
2
3
4
0
0
0
1
0
1
0
0
1
0
1
2
0
1
2
3
4
5
6
7
0
1
0
1
0
1
0
1
2
3
4
5
6
7
8
9
10
11
0
1
2
3
0
1
2
0
1
2
0
0
1
2
3
4
5
6
0
0
0
1
2
3
0
1
0
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
0
1
0
1
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
3